### Chain of Thought with RAG

Chain of Thoughts (CoT) in RAG ?
CoT reasoning breaks down a complex  question into intermediate steps and allow retrieval + reflection at each step before answering

- Step 1 : Decompose question -> Sub-steps(Reason)
- Step 2 : Retrieve docs per step (Act)
- Step 3 : Combine Context (Observe)
- Step 4 : Final Answer Generation (Reflect)

In [ ]:
import os
from dotenv import load_dotenv
from pydantic import BaseModel
from langchain_core.documents import Document
from langchain_community.document_loaders import TextLoader
from langchain_anthropic import ChatAnthropic
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS 
from langgraph.graph import StateGraph, END
from langchain_huggingface import HuggingFaceEmbeddings

In [ ]:
## Prepare VectoreStore
# .load() makes it a document
docs = TextLoader("Research_notes.txt", encoding = "utf-8").load()
chunks  = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap=50).split_documents(docs)
Embeddings = HuggingFaceEmbeddings(
    model_name  = "sentence-transformers/all-MiniLM-L6-v2"
)
vs = FAISS.from_documents(chunks, Embeddings)
retriever = vs.as_retriever()


In [18]:
load_dotenv()


api_key  = os.getenv("claudeAPI")

llm = ChatAnthropic(
    model = "claude-haiku-4-5-20251001",
    temperature = 0.7,
    api_key = api_key
)

In [24]:
from typing import Annotated, List
# List it's not a data, it's just used for Data Annotation. As in python variable are Dynamic, using pydantic we make Static
class RAGCoTAgent(BaseModel) :
    question : str
    sub_steps : List[str] = []
    retrieved_docs : List[Document] = []
    ans : str = ""

In [ ]:
### 3 Nodes
def plan_steps(state : RAGCoTAgent) -> RAGCoTAgent:
    prompt = """ 
     Break the question into 2-3 reasoning steps : \n\n {state.question}
    """
    result = llm.invoke(prompt).content
    subqs = [line.strip("- ") for line in result.split("\n") if line.strip()]
    return state.model_copy(update = {"sub_steps" : subqs})
